In [1]:
#import keras
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense,Dropout,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from keras.applications import Xception
import pandas as pd
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
#from sklearn.metrics import confusion_matrix

h=299
w=299
h_predict=299
w_predict=299
bs=16
E=100

name_dic = {'Black-grass': 0, 'Charlock': 1, 'Cleavers': 2, 'Common Chickweed': 3, 'Common wheat': 4,
                'Fat Hen': 5, 'Loose Silky-bent': 6, 'Maize': 7, 'Scentless Mayweed': 8, 'Shepherds Purse': 9,
                'Small-flowered Cranesbill': 10, 'Sugar beet': 11}
print(os.listdir("../input"))
print(os.listdir("../input/train"))
name_dic2 = {'0': 'Black-grass', '1': 'Charlock', '2': 'Cleavers',
                '3': 'Common Chickweed', '4': 'Common wheat',
                '5': 'Fat Hen', '6': 'Loose Silky-bent', '7': 'Maize',
                '8': 'Scentless Mayweed', '9': 'Shepherds Purse',
                '10': 'Small-flowered Cranesbill', '11': 'Sugar beet'}


Using TensorFlow backend.


['sample_submission.csv', 'train', 'test']
['Common Chickweed', 'Fat Hen', 'Loose Silky-bent', 'Small-flowered Cranesbill', 'Shepherds Purse', 'Maize', 'Sugar beet', 'Common wheat', 'Cleavers', 'Black-grass', 'Charlock', 'Scentless Mayweed']


In [2]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [3]:
def preprocess_img(path):
    images=[]   #存的是np数组的照片
    labels=[]   #存的是照片对应的one hot label
    for file_name in os.listdir(path):
        for img_name in os.listdir(path+file_name):
            image=cv2.imread(path+file_name+r'/'+img_name)
            #if image is not None:
            #image = segment_plant(image)
            #image = sharpen_image(image)
            image=cv2.resize(image,(h,w))       #?????
            images.append(image)#(path+file_name+r'/'+img_name)
            labels.append(int(name_dic[file_name]))
    images=np.reshape(images,(-1,h,w,3))#.astype(np.float32)/255.0
    labels=np.array(labels)
    return images,labels

In [4]:
def preprocess_img_predict(path):
    images=[]   #存的是np数组的照片
    labels_name=[]   #存的是照片对应的one hot label
    for file_name in os.listdir(path):
        #for img_name in os.listdir(path+file_name):
        image=cv2.imread(path+file_name)
        #if image is not None:
        #image = segment_plant(image)
        #image = sharpen_image(image)
        image=cv2.resize(image,(h_predict,w_predict))       #?????
        images.append(image)#(path+file_name+r'/'+img_name)
        labels_name.append(file_name)
    images=np.reshape(images,(-1,h_predict,w_predict,3))#.astype(np.float32)/255.0
    #labels_name=np.array(labels)  #不需要，后面需要是一个list
    return images,labels_name 

In [5]:
images,labels=preprocess_img('../input/train/')  #之前报错，是因为除以两次255.0

In [6]:
images_predict,labels_name=preprocess_img_predict('../input/test/')   

In [7]:
img_train, img_test, label_train, label_test = train_test_split(images, labels, test_size=0.2, shuffle=True)
#shuffle默认true

In [8]:
label_train_onehot = to_categorical(label_train, 12)
label_test_onehot = to_categorical(label_test, 12)

In [9]:
del labels
del images

In [10]:
def get_model(a,b):
    benchmark=Xception(weights='imagenet',input_shape=(a,b,3),include_top=False)
    x=benchmark.output
    #print(x.shape)
    x=GlobalAveragePooling2D()(x)
    x=Dropout(0.5)(x)
    x=Dense(1024,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(12, activation='softmax')(x)

    model=Model(inputs=benchmark.input,outputs=x)

    return model

In [11]:
def show_train_history(H):
    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    #N = E # 训练周期数
    N=len(H.epoch)
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
    plt.title("Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig('plot.png')
    plt.show()

In [12]:
train_datagen=ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True)
train_generator=train_datagen.flow(img_train,label_train_onehot,batch_size=bs)

#train_test_generator=train_datagen.flow(images,labels_onehot,batch_size=bs)

#filepath='../input/weights.best_{epoch:02d}-{val_acc:.2f}.hdf5'
rlr=ReduceLROnPlateau(monitor='val_acc',patience=3,factor=0.4,min_lr=0.00001,verbose=1)
#es=EarlyStopping(monitor='val_acc',patience=5)
#es=
#ckp=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True)
callbacks_list=[rlr]

In [13]:
def train_test():
    model=get_model(h,w)
    model.compile(optimizer='Adadelta',loss='categorical_crossentropy',metrics=['acc'])
    H=model.fit_generator(train_generator,steps_per_epoch=img_train.shape[0]//bs,epochs=E,validation_data=(img_test,label_test_onehot),callbacks=callbacks_list)   #len(img_train)//bs
    #img_predict=model.predict(img_test)
    #os.mkdir('model.h5')
    #model.save_weights('model.h5')
    #print('succeed save')
    #print(os.listdir("../input"))
    img_predict = model.predict(img_test)
    img_predict_onehot=np.argmax(img_predict,axis=1)

    matrix = pd.crosstab(label_test, img_predict_onehot, rownames=['label'], colnames=['predict'])
    print(matrix)

    show_train_history(H)
    #print('firt done')
    
    #HH=model.fit(img_test,label_test_onehot,epochs=3,batch_size=bs)
    model.save_weights('model.h5')
    #show_train_history(H)
    #print('succeed save')
    #print('second done')

In [14]:
if __name__=='__main__':
    train_test()
    #print('train finish')

Instructions for updating:
Colocations handled automatically by placer.
83689472/83683744 [==============================] - 6s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
237/237 [==============================] - 116s 490ms/step - loss: 0.9866 - acc: 0.6857 - val_loss: 0.4519 - val_acc: 0.8863
Epoch 2/100
237/237 [==============================] - 103s 436ms/step - loss: 0.4287 - acc: 0.8668 - val_loss: 0.5285 - val_acc: 0.8747
Epoch 3/100
237/237 [==============================] - 103s 435ms/step - loss: 0.3308 - acc: 0.8945 - val_loss: 0.2390 - val_acc: 0.9326
Epoch 4/100
237/237 [==============================] - 103s 435ms/step - loss: 0.2841 - acc: 0.9082 - val_loss: 0.2818 - val_acc: 0.9274
Epoch 5/100
237/237 [==============================] - 103s 435ms/step - loss: 0.2620 - acc: 0.9122 - val_loss: 0.2927 - val_acc: 0.9084
Epoch 6/100
237

In [15]:
def predict():
    model=get_model(h_predict,w_predict)   #h_predict,w_predict
    model.load_weights('model.h5')
    print('load succeed')
    p=model.predict(images_predict)
    p_onehot=np.argmax(p,axis=1)
    
    list=[]
    for i in p_onehot:
        list.append(name_dic2[str(i)])
    #print(len(label_name))
    #print(len(list))
    result=pd.DataFrame({'file':labels_name,'species':list})
    result.to_csv('myresult_noseg.csv',index=False)
    #print('finish')

if __name__=='__main__':
    predict()   
    #print('predict finish')

load succeed
